In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import functions
import math

In [ ]:
filepath ='data/'
csvfilenames = ['confinement_training.csv', 
                'member_information.csv', 
                'labresults_training.csv', 
                'medical_training.csv', 
                'rx_training.csv',
                'medical_target.csv',
                'rx_target.csv']

with open(filepath+csvfilenames[1]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

members = pd.DataFrame(data)
members.columns = members.iloc[0]
members = members[1:]
members = members.set_index(['PATID']) #makes PATID the row index instead of integers, not sure it's worth it.

with open(filepath+csvfilenames[2]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

labdata = pd.DataFrame(data)
labdata.columns = labdata.iloc[0]
labdata = labdata[1:]

with open(filepath+csvfilenames[3]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

allmedical = pd.DataFrame(data)
allmedical.columns = allmedical.iloc[0]
allmedical = allmedical[1:]

with open(filepath+csvfilenames[4]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

rxdata = pd.DataFrame(data)
rxdata.columns = rxdata.iloc[0]
rxdata = rxdata[1:]

with open(filepath+csvfilenames[5]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

targetallmedical = pd.DataFrame(data)
targetallmedical.columns = targetallmedical.iloc[0]
targetallmedical = targetallmedical[1:]

with open(filepath+csvfilenames[6]) as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)

targetrxdata = pd.DataFrame(data)
targetrxdata.columns = targetrxdata.iloc[0]
targetrxdata = targetrxdata[1:]

In [ ]:
medical_wanted = allmedical[('630' >  allmedical.DIAG1) | (allmedical.DIAG1 > '67999')]
medical_wanted = medical_wanted[('630' >  medical_wanted.DIAG2) | (medical_wanted.DIAG2 > '67999')]

medical_wanted = medical_wanted[('800' >  medical_wanted.DIAG1) | (medical_wanted.DIAG1 > '95999')]
medical_wanted = medical_wanted[('800' >  medical_wanted.DIAG2) | (medical_wanted.DIAG2 > '95999')]

medical_wanted = medical_wanted[medical_wanted['DIAG1'].str.startswith('E') == False]
medical_wanted = medical_wanted[medical_wanted['DIAG2'].str.startswith('E') == False]

medical_wanted = medical_wanted[(medical_wanted['DIAG1'].str.startswith('V3') == False) & (medical_wanted['DIAG1'].str.startswith('V22') == False) & (medical_wanted['DIAG1'].str.startswith('V23') == False) & (medical_wanted['DIAG1'].str.startswith('V24') == False) & (medical_wanted['DIAG1'].str.startswith('V28') == False) & (medical_wanted['DIAG1'].str.startswith('V91') == False)]
medical_wanted = medical_wanted[(medical_wanted['DIAG2'].str.startswith('V3') == False) & (medical_wanted['DIAG2'].str.startswith('V22') == False) & (medical_wanted['DIAG2'].str.startswith('V23') == False) & (medical_wanted['DIAG2'].str.startswith('V24') == False) & (medical_wanted['DIAG2'].str.startswith('V28') == False) & (medical_wanted['DIAG2'].str.startswith('V91') == False)]



medical_wanted_t = targetallmedical[('630' >  targetallmedical.DIAG1) | (targetallmedical.DIAG1 > '67999')]
medical_wanted_t = medical_wanted_t[('630' >  medical_wanted_t.DIAG2) | (medical_wanted_t.DIAG2 > '67999')]

medical_wanted_t = medical_wanted_t[('800' >  medical_wanted_t.DIAG1) | (medical_wanted_t.DIAG1 > '95999')]
medical_wanted_t = medical_wanted_t[('800' >  medical_wanted_t.DIAG2) | (medical_wanted_t.DIAG2 > '95999')]

medical_wanted_t = medical_wanted_t[medical_wanted_t['DIAG1'].str.startswith('E') == False]
medical_wanted_t = medical_wanted_t[medical_wanted_t['DIAG2'].str.startswith('E') == False]

medical_wanted_t = medical_wanted_t[(medical_wanted_t['DIAG1'].str.startswith('V3') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V22') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V23') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V24') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V28') == False) & 
                                    (medical_wanted_t['DIAG1'].str.startswith('V91') == False)]
medical_wanted_t = medical_wanted_t[(medical_wanted_t['DIAG2'].str.startswith('V3') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V22') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V23') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V24') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V28') == False) & 
                                    (medical_wanted_t['DIAG2'].str.startswith('V91') == False)]

In [ ]:
#medical_wanted.to_csv('trainingmedicalwanted.csv')
#medical_wanted_t.to_csv('targetmedicalwanted.csv')

In [ ]:
medical_wanted['STD_COST']=medical_wanted['STD_COST'].apply(pd.to_numeric) #change entries to numbers
rxdata.loc[rxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
rxdata['STD_COST']=rxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
members['year_of_birth']=members['year_of_birth'].apply(pd.to_numeric)
rxdata['DAYS_FROM_DIAG'] = rxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
medical_wanted['DAYS_FROM_DIAG'] = medical_wanted['DAYS_FROM_DIAG'].apply(pd.to_numeric)

medical_wanted_t['STD_COST']=medical_wanted_t['STD_COST'].apply(pd.to_numeric) #change entries to numbers
targetrxdata.loc[targetrxdata['STD_COST']=='None', 'STD_COST']=0 #change all nones to zeros
targetrxdata['STD_COST']=targetrxdata['STD_COST'].apply(pd.to_numeric) #change all entries to numbers
targetrxdata['DAYS_FROM_DIAG'] = targetrxdata['DAYS_FROM_DIAG'].apply(pd.to_numeric)
medical_wanted_t['DAYS_FROM_DIAG'] = medical_wanted_t['DAYS_FROM_DIAG'].apply(pd.to_numeric)

In [ ]:
#add training data
members['totalcost'] = 0.0
members['rxcost'] = 0.0
members['medicalcost'] = 0.0

dummy = rxdata.groupby('PATID')['STD_COST'].sum()

for k in range(len(dummy)):
    members.at[dummy.index[k],'totalcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'rxcost'] = dummy.iat[k]
    
dummy = medical_wanted.groupby('PATID')['STD_COST'].sum()

for k in range(len(dummy)):
    members.at[dummy.index[k], 'totalcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'medicalcost'] = dummy.iat[k]
    

#add target data
members['targetcost'] = 0.0 
members['targetrxcost'] = 0.0
members['targetmedicalcost'] = 0.0

dummy = targetrxdata.groupby('PATID')['STD_COST'].sum()


for k in range(len(dummy)):
    members.at[dummy.index[k],'targetcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'targetrxcost'] = dummy.iat[k]

    
dummy = medical_wanted_t.groupby('PATID')['STD_COST'].sum()


for k in range(len(dummy)):
    members.at[dummy.index[k], 'targetcost'] += dummy.iat[k]
    members.at[dummy.index[k], 'targetmedicalcost'] = dummy.iat[k]

In [ ]:
#all potentially completely useless
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minrxday'] = dummy.iat[k]
dummy = rxdata.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxrxday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].min()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'minmedicalday'] = dummy.iat[k]
dummy = medical_wanted.groupby('PATID')['DAYS_FROM_DIAG'].max()
for k in range(len(dummy)):
    members.at[dummy.index[k], 'maxmedicalday'] = dummy.iat[k]

In [ ]:
#Split costs into prediagnosis and postdiagnosis
    members['prediag_medicalcost'] = 0.0
    members['postdiag_medicalcost'] = 0.0
    members['prediag_rxcost'] = 0.0
    members['postdiag_rxcost'] = 0.0
    members['prediag_totalcost'] = 0.0
    members['postdiag_totalcost'] = 0.0

    prediag_med = medical_wanted[medical_wanted.DAYS_FROM_DIAG>=0].groupby('PATID')['STD_COST'].sum()
    postdiag_med = medical_wanted[medical_wanted.DAYS_FROM_DIAG<0].groupby('PATID')['STD_COST'].sum()

    prediag_rx = rxdata[rxdata.DAYS_FROM_DIAG>=0].groupby('PATID')['STD_COST'].sum()
    postdiag_rx = rxdata[rxdata.DAYS_FROM_DIAG<0].groupby('PATID')['STD_COST'].sum()

    for k in range(len(prediag_med)):
        members.at[prediag_med.index[k], 'prediag_medicalcost'] = prediag_med.iat[k]
        members.at[prediag_med.index[k], 'prediag_totalcost'] += prediag_med.iat[k]
    for k in range(len(postdiag_med)):
        members.at[postdiag_med.index[k], 'postdiag_medicalcost'] = postdiag_med.iat[k]
        members.at[postdiag_med.index[k], 'postdiag_totalcost'] += postdiag_med.iat[k]
    for k in range(len(prediag_rx)):
        members.at[prediag_rx.index[k], 'prediag_rxcost'] = prediag_rx.iat[k]
        members.at[prediag_rx.index[k], 'prediag_totalcost'] += prediag_rx.iat[k]
    for k in range(len(postdiag_rx)):
        members.at[postdiag_rx.index[k], 'postdiag_rxcost'] = postdiag_rx.iat[k]
        members.at[postdiag_rx.index[k], 'postdiag_totalcost'] += postdiag_rx.iat[k]

    #Calculate average daily costs
    members['prediag_days'] = 0       # First need to calculate number of days before and after diagnosis
    members['postdiag_days'] = 0

    members['prediag_days'] = -1 * members[['minmedicalday', 'minrxday']].min(axis=1) + 1
    members['postdiag_days'] = members[['maxmedicalday', 'maxrxday']].max(axis=1) + 1

    members['prediag_dailymedcost'] = members['prediag_medicalcost'] / members['prediag_days']
    members['postdiag_dailymedcost'] = members['postdiag_medicalcost'] / members['postdiag_days']
    members['prediag_dailyrxcost'] = members['prediag_rxcost'] / members['prediag_days']
    members['postdiag_dailyrxcost'] = members['postdiag_rxcost'] / members['postdiag_days']
    members['prediag_dailytotalcost'] = members['prediag_totalcost'] / members['prediag_days']
    members['postdiag_dailytotalcost'] = members['postdiag_totalcost'] / members['postdiag_days']


    members['dailymedicalcost'] = members['medicalcost'] / (members['prediag_days'] + members['postdiag_days'])
    members['dailyrxcost'] = members['rxcost'] / (members['prediag_days'] + members['postdiag_days'])
    members['dailytotalcost'] = members['totalcost'] / (members['prediag_days'] + members['postdiag_days'])

In [ ]:
#emergency=['23','24', '21', '41', '20', '42']
medical_wanted_emergency = medical_wanted[(medical_wanted['POS'] =='23') | 
                                          (medical_wanted['POS'] =='24') | 
                                          (medical_wanted['POS'] =='21') |
                                          (medical_wanted['POS'] =='41') |
                                          (medical_wanted['POS'] =='20') |
                                          (medical_wanted['POS'] =='42') ]
members['emervisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_emergency.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'emervisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#psychiatric=['53', '55', '51', '52', '57'] # 51 is inpatient faciliity, 55 is res. sub. abuse
medical_wanted_psych = medical_wanted[(medical_wanted['POS'] =='53') | 
                                          #(medical_wanted['POS'] =='55') |
                                          (medical_wanted['POS'] =='52') |
                                          (medical_wanted['POS'] =='57')]
members['psychvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_psych.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'psychvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#regular=['11', '01', '81','22', '60', '49', '71', '62', '17', '50', '61', '03']
medical_wanted_office = medical_wanted[(medical_wanted['POS'] =='11') | 
                                          (medical_wanted['POS'] =='01') | 
                                          (medical_wanted['POS'] =='81') |
                                          (medical_wanted['POS'] =='22') |
                                          (medical_wanted['POS'] =='60') |
                                      (medical_wanted['POS'] =='49') | 
                                          (medical_wanted['POS'] =='71') |
                                          (medical_wanted['POS'] =='62') |
                                          (medical_wanted['POS'] =='17') |
                                      (medical_wanted['POS'] =='50') | 
                                          (medical_wanted['POS'] =='61') |
                                          (medical_wanted['POS'] =='03') ]
members['officevisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_office.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'officevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#elderly=['13', '31', '32', '33']
medical_wanted_elderly = medical_wanted[(medical_wanted['POS'] =='13') | 
                                          (medical_wanted['POS'] =='31') | 
                                          (medical_wanted['POS'] =='32') |
                                          (medical_wanted['POS'] =='33') ]
members['elderlyvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_elderly.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'elderlyvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#home=['12', '14', '15']
medical_wanted_home = medical_wanted[(medical_wanted['POS'] =='12') | 
                                          (medical_wanted['POS'] =='14') | 
                                          (medical_wanted['POS'] =='15')]
members['homevisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_home.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'homevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#dialysis=['65']
medical_wanted_dialysis = medical_wanted[(medical_wanted['POS'] =='65')]
members['dialysisvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_dialysis.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'dialysisvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#hospice=['34']
medical_wanted_hospice = medical_wanted[(medical_wanted['POS'] =='34')]
members['hospicevisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_hospice.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'hospicevisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#unknown=['99', 'None']
medical_wanted_unknown = medical_wanted[(medical_wanted['POS'] =='99')|
                                        (medical_wanted['POS'] == 'None')]
members['unknownvisits']=0
patids = list(members.index)
tick = time.clock()
patidgroups = medical_wanted_unknown.groupby('PATID')
for k in patids:
    try:
        members.loc[k, 'unknownvisits'] = len(patidgroups.get_group(k).groupby('DAYS_FROM_DIAG'))
    except KeyError:
        pass
tock = time.clock()
print tock-tick

In [ ]:
#Plot average cost for all POS codes
location_series = medical_wanted.groupby('POS')['STD_COST'].mean()

location_series.plot.bar()
plt.ylim([0,2000])
plt.show()

#42-Ambulance
#21- Inpatient Hospital
#22- Outpatient Hospital
#24- Ambulatory Surgical Center
#31- Skilled Nursing
#65- End stage renal disease facility

In [ ]:
# Add new column for region (should find a way to avoid this with groupby, but can't currently)
members['region'] = members.apply(lambda row: functions.US_region(row), axis=1)

region_series = members.groupby('region')['totalcost'].mean()

region_series.plot.bar()
plt.show()

In [ ]:
#members.to_csv('members.csv')

In [ ]:
#The next series of cells creates bins for the rx data

In [ ]:
#Create 5-day bins and bin labels. I think this puts bin 0 at bin 0,5
print rxdata.DAYS_FROM_DIAG.max() #Find max days from diagnosis
print rxdata.DAYS_FROM_DIAG.min() #Find minimum days before diagnosis
min_day = rxdata.DAYS_FROM_DIAG.min()
max_day = rxdata.DAYS_FROM_DIAG.max()
#Compute the number of bins at some point, didn't automate this
rxdata['fivedaybin'] = pd.cut(rxdata['DAYS_FROM_DIAG'], range(-730, 1100, 5), right=False) #bin by 5 days rounding on the outside to nice numbers
r5=range(-730,1100+5,5) #define bin edges
binlabel5=range(-147,219) #number of negative bins, number of positive bins. Remembering how range works means it should put bin 0 in the correct spot
#categories = pd.cut(rxdata['DAYS_FROM_DIAG'], r, labels=binlabel) #run this if you want to see the bin labels
#print categories
rxdata['fivedaybin_id'] = pd.cut(rxdata['DAYS_FROM_DIAG'], r5, labels=binlabel5)
#print rxdata.head(n=3)

In [ ]:
#Create 7-day bins and bin labels.
rxdata['sevendaybin']=pd.cut(rxdata['DAYS_FROM_DIAG'],range(-728,1099,7),right=False) #create the bins
numbins = (728+1099)/7 #automate endpoint search? Probably not worth it
numneg = -728/7
numpos=1099/7
print numneg
print numpos
r7=range(-728,1099+7,7)
binlabel7=range(numneg,numpos)
rxdata['sevendaybin_id'] = pd.cut(rxdata['DAYS_FROM_DIAG'], r7, labels=binlabel7)
#print rxdata.head(n=3)

In [ ]:
#Create 10-day bins and bin labels.
rxdata['tendaybin']=pd.cut(rxdata['DAYS_FROM_DIAG'],range(-730,1100,10),right=False) #create the bins
numbins = (730+1100)/10 #automate endpoint search? Probably not worth it
numneg = -730/10
numpos=1100/10
print numneg
print numpos
r10=range(-730,1100+10,10)
binlabel10=range(numneg,numpos)
rxdata['tendaybin_id'] = pd.cut(rxdata['DAYS_FROM_DIAG'], r10, labels=binlabel10)
#print rxdata.head(n=3)

In [ ]:
#Create 20-day bins and bin labels.
rxdata['twentydaybin']=pd.cut(rxdata['DAYS_FROM_DIAG'],range(-740,1100,20),right=False) #create the bins
numbins = (740+1100)/20 #automate endpoint search? Probably not worth it
numneg = -740/20
numpos=1100/20
print numneg
print numpos
r20=range(-740,1100+20,20)
binlabel20=range(numneg,numpos)
rxdata['twentydaybin_id'] = pd.cut(rxdata['DAYS_FROM_DIAG'], r20, labels=binlabel20)
#print rxdata.head(n=3)

In [ ]:
#Create 30-day bins and bin labels.
binsize = 30
numbins = int(math.ceil(-min_day/binsize) + math.ceil(max_day/binsize)) #automate endpoint search? Probably not worth it
numneg = int(math.ceil(min_day/binsize))
numpos = int(math.ceil(max_day/binsize))
print numneg
print numpos
r30=range(numneg*binsize,(numpos+1)*binsize,binsize)
binlabel30=range(numneg,numpos)
rxdata['thirtydaybin']=pd.cut(rxdata['DAYS_FROM_DIAG'],r30,right=False) #create the bins
rxdata['thirtydaybin_id'] = pd.cut(rxdata['DAYS_FROM_DIAG'], r30, labels=binlabel30)
#print rxdata.head(n=3)

In [ ]:
#Create 60-day bins and bin labels.
binsize = 60
numbins = int(math.ceil(-min_day/binsize) + math.ceil(max_day/binsize)) #automate endpoint search? Probably not worth it
numneg = int(math.ceil(min_day/binsize))
numpos = int(math.ceil(max_day/binsize))
print numneg
print numpos
r30=range(numneg*binsize,(numpos+1)*binsize,binsize)
binlabel30=range(numneg,numpos)
rxdata['sixtydaybin']=pd.cut(rxdata['DAYS_FROM_DIAG'],r30,right=False) #create the bins
rxdata['sixtydaybin_id'] = pd.cut(rxdata['DAYS_FROM_DIAG'], r30, labels=binlabel30)
#print rxdata.head(n=3)

In [ ]:
#Create 120-day bins and bin labels.
binsize = 120
numbins = int(math.ceil(-min_day/binsize) + math.ceil(max_day/binsize)) #automate endpoint search? Probably not worth it
numneg = int(math.ceil(min_day/binsize))
numpos = int(math.ceil(max_day/binsize))
print numneg
print numpos
r30=range(numneg*binsize,(numpos+1)*binsize,binsize)
binlabel30=range(numneg,numpos)
rxdata['onetwentydaybin']=pd.cut(rxdata['DAYS_FROM_DIAG'],r30,right=False) #create the bins
rxdata['onetwentydaybin_id'] = pd.cut(rxdata['DAYS_FROM_DIAG'], r30, labels=binlabel30)
#print rxdata.head(n=3)

In [ ]:
temp = rxdata.groupby(['PATID', 'sixtydaybin_id'])['STD_COST'].sum().reset_index(name='bin_cost')

temp.groupby(['sixtydaybin_id'])['bin_cost'].mean().plot()
plt.show()

In [ ]:
#the next series of cells creates bins for the medical_wanted data

In [ ]:
min_day_med = medical_wanted.DAYS_FROM_DIAG.min()
max_day_med = medical_wanted.DAYS_FROM_DIAG.max()
print min_day_med
print max_day_med

In [ ]:
#Create 5-day bins and bin labels.
binsize = 5
numbins = int(math.ceil(-min_day_med/binsize) + math.ceil(max_day_med/binsize)) #automate endpoint search? Probably not worth it
numneg = int(math.ceil(min_day_med/binsize))
numpos = int(math.ceil(max_day_med/binsize))
print numneg
print numpos
r=range(numneg*binsize,(numpos+1)*binsize,binsize)
binlabel=range(numneg,numpos)
medical_wanted['fivedaybin']=pd.cut(medical_wanted['DAYS_FROM_DIAG'],r,right=False) #create the bins
medical_wanted['fivedaybin_id'] = pd.cut(medical_wanted['DAYS_FROM_DIAG'], r, labels=binlabel)
#print medical_wanted.head(n=3)

In [ ]:
temp = medical_wanted.groupby(['PATID', 'fivedaybin_id'])['STD_COST'].sum().reset_index(name='bin_cost')

temp.groupby(['fivedaybin_id'])['bin_cost'].mean().plot()
plt.show()

In [ ]:
#Create 7-day bins and bin labels.
binsize = 7
numbins = int(math.ceil(-min_day_med/binsize) + math.ceil(max_day_med/binsize)) #automate endpoint search? Probably not worth it
numneg = int(math.ceil(min_day_med/binsize))
numpos = int(math.ceil(max_day_med/binsize))
print numneg
print numpos
r=range(numneg*binsize,(numpos+1)*binsize,binsize)
binlabel=range(numneg,numpos)
medical_wanted['sevendaybin']=pd.cut(medical_wanted['DAYS_FROM_DIAG'],r,right=False) #create the bins
medical_wanted['sevendaybin_id'] = pd.cut(medical_wanted['DAYS_FROM_DIAG'], r, labels=binlabel)
#print medical_wanted.head(n=3)

In [ ]:
temp = medical_wanted.groupby(['PATID', 'sevendaybin_id'])['STD_COST'].sum().reset_index(name='bin_cost')

temp.groupby(['sevendaybin_id'])['bin_cost'].mean().plot()
plt.show()

In [ ]:
#Create 10-day bins and bin labels.
binsize = 10
numbins = int(math.ceil(-min_day_med/binsize) + math.ceil(max_day_med/binsize)) #automate endpoint search? Probably not worth it
numneg = int(math.ceil(min_day_med/binsize))
numpos = int(math.ceil(max_day_med/binsize))
print numneg
print numpos
r=range(numneg*binsize,(numpos+1)*binsize,binsize)
binlabel=range(numneg,numpos)
medical_wanted['tendaybin']=pd.cut(medical_wanted['DAYS_FROM_DIAG'],r,right=False) #create the bins
medical_wanted['tendaybin_id'] = pd.cut(medical_wanted['DAYS_FROM_DIAG'], r, labels=binlabel)
#print medical_wanted.head(n=3)

In [ ]:
temp = medical_wanted.groupby(['PATID', 'tendaybin_id'])['STD_COST'].sum().reset_index(name='bin_cost')

temp.groupby(['tendaybin_id'])['bin_cost'].mean().plot()
plt.show()

In [ ]:
#Create 20-day bins and bin labels.
binsize = 20
numbins = int(math.ceil(-min_day_med/binsize) + math.ceil(max_day_med/binsize)) #automate endpoint search? Probably not worth it
numneg = int(math.ceil(min_day_med/binsize))
numpos = int(math.ceil(max_day_med/binsize))
print numneg
print numpos
r=range(numneg*binsize,(numpos+1)*binsize,binsize)
binlabel=range(numneg,numpos)
medical_wanted['twentydaybin']=pd.cut(medical_wanted['DAYS_FROM_DIAG'],r,right=False) #create the bins
medical_wanted['twentydaybin_id'] = pd.cut(medical_wanted['DAYS_FROM_DIAG'], r, labels=binlabel)
#print medical_wanted.head(n=3)

In [ ]:
temp = medical_wanted.groupby(['PATID', 'twentydaybin_id'])['STD_COST'].sum().reset_index(name='bin_cost')

temp.groupby(['twentydaybin_id'])['bin_cost'].mean().plot()
plt.show()

In [ ]:
#Create 30-day bins and bin labels.
binsize = 30
numbins = int(math.ceil(-min_day_med/binsize) + math.ceil(max_day_med/binsize)) #automate endpoint search? Probably not worth it
numneg = int(math.ceil(min_day_med/binsize))
numpos = int(math.ceil(max_day_med/binsize))
print numneg
print numpos
r=range(numneg*binsize,(numpos+1)*binsize,binsize)
binlabel=range(numneg,numpos)
medical_wanted['thirtydaybin']=pd.cut(medical_wanted['DAYS_FROM_DIAG'],r,right=False) #create the bins
medical_wanted['thirtydaybin_id'] = pd.cut(medical_wanted['DAYS_FROM_DIAG'], r, labels=binlabel)
#print medical_wanted.head(n=3)

In [ ]:
temp = medical_wanted.groupby(['PATID', 'thirtydaybin_id'])['STD_COST'].sum().reset_index(name='bin_cost')

temp.groupby(['thirtydaybin_id'])['bin_cost'].mean().plot()
plt.show()

In [ ]:
#Create 60-day bins and bin labels.
binsize = 60
numbins = int(math.ceil(-min_day_med/binsize) + math.ceil(max_day_med/binsize)) #automate endpoint search? Probably not worth it
numneg = int(math.ceil(min_day_med/binsize))
numpos = int(math.ceil(max_day_med/binsize))
print numneg
print numpos
r=range(numneg*binsize,(numpos+1)*binsize,binsize)
binlabel=range(numneg,numpos)
medical_wanted['sixtydaybin']=pd.cut(medical_wanted['DAYS_FROM_DIAG'],r,right=False) #create the bins
medical_wanted['sixtydaybin_id'] = pd.cut(medical_wanted['DAYS_FROM_DIAG'], r, labels=binlabel)
#print medical_wanted.head(n=3)

In [ ]:
temp = medical_wanted.groupby(['PATID', 'sixtydaybin_id'])['STD_COST'].sum().reset_index(name='bin_cost')

temp.groupby(['sixtydaybin_id'])['bin_cost'].mean().plot()
plt.show()

In [ ]:
#Create 120-day bins and bin labels.
binsize = 120
numbins = int(math.ceil(-min_day_med/binsize) + math.ceil(max_day_med/binsize)) #automate endpoint search? Probably not worth it
numneg = int(math.ceil(min_day_med/binsize))
numpos = int(math.ceil(max_day_med/binsize))
print numneg
print numpos
r=range(numneg*binsize,(numpos+1)*binsize,binsize)
binlabel=range(numneg,numpos)
medical_wanted['onetwentydaybin']=pd.cut(medical_wanted['DAYS_FROM_DIAG'],r,right=False) #create the bins
medical_wanted['onetwentydaybin_id'] = pd.cut(medical_wanted['DAYS_FROM_DIAG'], r, labels=binlabel)
#print medical_wanted.head(n=3)

In [ ]:
temp = medical_wanted.groupby(['PATID', 'onetwentydaybin_id'])['STD_COST'].sum().reset_index(name='bin_cost')

temp.groupby(['onetwentydaybin_id'])['bin_cost'].mean().plot()
plt.show()

In [ ]:
#try to find out what's going on with spike
temp2=medical_wanted.groupby('fivedaybin_id')['STD_COST'].sum()
temp2=temp2.sort_values(ascending=False)
#print temp2

#Bin label 205 is giving the crazy spike

In [ ]:
temp = medical_wanted.groupby(['PATID', 'fivedaybin_id'])['STD_COST'].sum().reset_index(name='bin_cost')

temp2=temp.groupby(['fivedaybin_id'])['bin_cost'].mean()

temp2=temp2.sort_values(ascending=False)

print temp.head(n=3)
print temp2

In [ ]:
#temp.loc[temp['fivedaybin_id']==205]

In [ ]:
expensive_guy=medical_wanted.loc[medical_wanted['PATID']=='33068627845.0']
eg_bin=expensive_guy.loc[expensive_guy['fivedaybin_id']==205]
print eg_bin.DIAG1.unique()
#disease of tricuspid valve, viral pneumonia, hypoxemia, septicemia. Poor guy.

In [ ]:
#This cell shows an example where the total cost doesn't cancel to zero. You can see that it looks like maybe
#something got duplicated (look for negative entries and matching positive entries)
#print members_sorted.loc['33022788647.0']
medical_wanted[medical_wanted['PATID']=='33022788647.0']#Divide members into 5 bins with equal cost
members_sorted=members.sort_values('totalcost',ascending=False)
members_sorted['cum_totalcost']=members_sorted['totalcost'].expanding(1).sum() #compute the cumulative cost
cost_per_bin=members_sorted.totalcost.sum()/5 #compute total cost divided by desired number of bins
bins = [0, cost_per_bin, 2*cost_per_bin, 3*cost_per_bin, 4*cost_per_bin, 5*cost_per_bin] #create bin edges
quintiles=[5,4,3,2,1] #'name' the bins
members_sorted['quintile']=pd.cut(members_sorted['cum_totalcost'],bins,right=False) #create the bins
members_sorted['quintile_id'] = pd.cut(members_sorted['cum_totalcost'], bins, labels=quintiles)
print members_sorted.groupby('quintile_id')['totalcost'].sum() #check that we have approximately equal cost in each bin

In [ ]:
#Find the total number of members in each bucket for training
print members_sorted.groupby('quintile_id')['totalcost'].count()

In [ ]:
#Find the min and max cost in each bucket for training
maxlist=members_sorted.groupby('quintile_id')['totalcost'].max()
minlist=members_sorted.groupby('quintile_id')['totalcost'].min() #Somehow there is a negative cost?
print maxlist
print minlist
#Bins are -4508.80-17834.01, 17835.00-38323.95, 38330.14-76880.00, 76996.64-187896.19, 188112.68-5304105.38

In [ ]:
#Create bucket labels for target total cost
bins=(members_sorted.targetcost.min(),minlist[2],minlist[3],minlist[4],minlist[5],10000000)
print bins

In [ ]:
#why is there a negative minimum? Should we throw these out? How to deal with them?
print members_sorted['totalcost'].min()
print members_sorted['totalcost'].idxmin()

In [ ]:
medical_wanted_t[medical_wanted_t['STD_COST']<0]
#It looks like there is usually an exact positive match to any positive number
#Don't really know what we should do about these. Throw them out? Tend to think there's duplicated causing non-cancellation

In [ ]:
#This cell shows an example where the total cost doesn't cancel to zero. You can see that it looks like maybe
#something got duplicated (look for negative entries and matching positive entries)
#print members_sorted.loc['33022788647.0']
medical_wanted[medical_wanted['PATID']=='33022788647.0']